In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nltk, os
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Arbeitsverzeichnis auf das Ordnerverzeichnis ändern, in welchem die Daten liegen (txt-Format)

#cwd = os.getcwd()
nwd = r'/Users/paul/Documents/FOM/Analyse semi- und unstrukturierter Daten/CAPTUM_txt'
os.chdir(nwd)

#Speichern der Dateinamen in einem Array
files = os.listdir()

In [3]:
#Zu Testzwecken werden lediglich die letzten 5 Dateien verwendet
files = files[100:201]

In [4]:
fileNameTags = list(map(lambda fn: fn.split('_')[0:-2],files))

fileName = [fNT[-1] for fNT in fileNameTags]
fileTags = [fNT[1:-1] for fNT in fileNameTags]
fileYear = [fY[-8:-4] for fY in fileName]

print(fileNameTags)
print(fileName)
print(fileTags)
print(fileYear)

[['CAPTUM', 'Allergic Diseases', 'Antihistamine', 'Maurer 2017 .pdf'], ['CAPTUM', 'Antihistamine', 'Omalizumab', 'Giménez‑Arnau 2020.pdf'], ['CAPTUM', 'ANA', 'Omalizumab', 'Asero 2017.pdf'], ['CAPTUM', 'Total IgE', 'TPO', 'Aversano 2005.pdf'], ['CAPTUM', 'ASST\uf028', 'angioedema', 'Vikramkumar 2015.pdf'], ['CAPTUM', 'Total IgE', 'Allergic Diseases', 'Baker 2015.pdf'], ['CAPTUM', 'Allergic Diseases', 'Omalizumab', 'Nam 2012.pdf'], ['CAPTUM', 'ASST\uf028', 'Anti-IgE', 'Guttman-Yassky 2007.pdf'], ['CAPTUM', 'BAT', 'Anti-IgE', 'Oda 2019.pdf'], ['CAPTUM', 'Total IgE', 'Omalizumab', 'Syrigos 2018.pdf'], ['CAPTUM', 'Basophil', 'Antihistamine', '~Irinyi 2007.pdf'], ['CAPTUM', 'Severity', 'Angioedema', 'Kahveci 2020.pdf'], ['CAPTUM', 'Antihistamine', 'Omalizumab', 'Rottem 2014.pdf'], ['CAPTUM', 'Severity', 'Duration', 'Deza 2017.pdf'], ['CAPTUM', 'Basophil', 'Omalizumab', 'Acer 2018.pdf'], ['CAPTUM', 'Total IgE', 'Duration', 'Kessel 2010.pdf'], ['CAPTUM', 'Duration', 'Omalizumab', 'Turk 2016.

In [5]:
#Einen leeren Pandas-Dataframe erstellen

df = pd.DataFrame(index=files)
print(df)

Empty DataFrame
Columns: []
Index: [CAPTUM_Allergic Diseases_Antihistamine_Maurer 2017 .pdf_out_text.txt, CAPTUM_Antihistamine_Omalizumab_Giménez‑Arnau 2020.pdf_out_text.txt, CAPTUM_ANA_Omalizumab_Asero 2017.pdf_out_text.txt, CAPTUM_Total IgE_TPO_Aversano 2005.pdf_out_text.txt, CAPTUM_ASST_angioedema_Vikramkumar 2015.pdf_out_text.txt, CAPTUM_Total IgE_Allergic Diseases_Baker 2015.pdf_out_text.txt, CAPTUM_Allergic Diseases_Omalizumab_Nam 2012.pdf_out_text.txt, CAPTUM_ASST_Anti-IgE_Guttman-Yassky 2007.pdf_out_text.txt, CAPTUM_BAT_Anti-IgE_Oda 2019.pdf_out_text.txt, CAPTUM_Total IgE_Omalizumab_Syrigos 2018.pdf_out_text.txt, CAPTUM_Basophil_Antihistamine_~Irinyi 2007.pdf_out_text.txt, CAPTUM_Severity_Angioedema_Kahveci 2020.pdf_out_text.txt, CAPTUM_Antihistamine_Omalizumab_Rottem 2014.pdf_out_text.txt, CAPTUM_Severity_Duration_Deza 2017.pdf_out_text.txt, CAPTUM_Basophil_Omalizumab_Acer 2018.pdf_out_text.txt, CAPTUM_Total IgE_Duration_Kessel 2010.pdf_out_text.txt, CAPTUM_Duration_Omalizu

In [6]:
c = []
for file in files:
    f = open(file, "r", encoding = "ISO-8859-1")
    c.append(f.read())
    f.close

In [7]:
df['Name'] = fileName
df['Tags'] = fileTags
df['Year'] = fileYear
df['Content'] = c
df

,Name,Tags,Year,Content
CAPTUM_Allergic Diseases_Antihistamine_Maurer 2017 .pdf_out_text.txt,Maurer 2017 .pdf,"[Allergic Diseases, Antihistamine]",017,PROFESSOR FRANZISKA RUEFF (Orcid ID : 0000-000...
CAPTUM_Antihistamine_Omalizumab_Giménez‑Arnau 2020.pdf_out_text.txt,Giménez‑Arnau 2020.pdf,"[Antihistamine, Omalizumab]",2020,Drugs\nhttps://doi.org/10.1007/s40265-020-0138...
CAPTUM_ANA_Omalizumab_Asero 2017.pdf_out_text.txt,Asero 2017.pdf,"[ANA, Omalizumab]",2017,Clinical Allergology - Original Paper\n\nInter...
CAPTUM_Total IgE_TPO_Aversano 2005.pdf_out_text.txt,Aversano 2005.pdf,"[Total IgE, TPO]",2005,Allergy 2005: 60: 489-493\n\nOriginal article\...
CAPTUM_ASST_angioedema_Vikramkumar 2015.pdf_out_text.txt,Vikramkumar 2015.pdf,"[ASST, angioedema]",2015,11/14/2020 Autologous serum skin test as an in...
...,...,...,...,...
CAPTUM_Angioedema_Omalizumab_Maoz-Segal 2020.pdf_out_text.txt,Maoz-Segal 2020.pdf,"[Angioedema, Omalizumab]",2020,Maoz-Segal et al. World Allergy Organization J...
CAPTUM_Total IgE_TPO_Sanchez 2019.pdf_out_text.txt,Sanchez 2019.pdf,"[Total IgE, TPO]",2019,Allergy Asthma Immunol Res. 2019 Jan;11(1):29-...
CAPTUM_TPO_Omalizumab_Maurer 2011.pdf_out_text.txt,Maurer 2011.pdf,"[TPO, Omalizumab]",2011,Efficacy and safety of omalizumab in patients ...
CAPTUM_Antihistamine_Omalizumab_Gimenez Arnau 2016.pdf_out_text.txt,Gimenez Arnau 2016.pdf,"[Antihistamine, Omalizumab]",2016,Copyright © 2017 John Libbey Eurotext. Downloa...


In [8]:
stop_words=set(stopwords.words("english"))
def stoppingFunc(text, stop_words=stop_words):
    filtered_text=[]
    for word in text:
        if len(word) < 3:
            next
        elif sum(c.isdigit() for c in word)>0:
            next
        elif word not in stop_words:
            filtered_text.append(word)
        
    return filtered_text

In [9]:
def stemmingFunc(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in text]

In [10]:
def lemmatizingFunc(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text]

In [11]:
df['Content'] = df['Content'].apply(lambda text: nltk.word_tokenize(text))
df['Content'] = df['Content'].apply(stoppingFunc)
df['Content'] = df['Content'].apply(stemmingFunc)
df['Content'] = df['Content'].apply(lemmatizingFunc)
df['Corpus'] = [' '.join(x) for x in df['Content']]

In [13]:
df

,Name,Tags,Year,Content,Corpus
CAPTUM_Allergic Diseases_Antihistamine_Maurer 2017 .pdf_out_text.txt,Maurer 2017 .pdf,"[Allergic Diseases, Antihistamine]",017,"[professor, franziska, rueff, orcid, receiv, d...",professor franziska rueff orcid receiv date re...
CAPTUM_Antihistamine_Omalizumab_Giménez‑Arnau 2020.pdf_out_text.txt,Giménez‑Arnau 2020.pdf,"[Antihistamine, Omalizumab]",2020,"[drug, http, lead, articl, check, updat, targe...",drug http lead articl check updat target thera...
CAPTUM_ANA_Omalizumab_Asero 2017.pdf_out_text.txt,Asero 2017.pdf,"[ANA, Omalizumab]",2017,"[clinic, allergolog, origin, paper, intern, ar...",clinic allergolog origin paper intern archiv i...
CAPTUM_Total IgE_TPO_Aversano 2005.pdf_out_text.txt,Aversano 2005.pdf,"[Total IgE, TPO]",2005,"[allergi, origin, articl, copyright, blackwel,...",allergi origin articl copyright blackwel munks...
CAPTUM_ASST_angioedema_Vikramkumar 2015.pdf_out_text.txt,Vikramkumar 2015.pdf,"[ASST, angioedema]",2015,"[autolog, serum, skin, test, indic, chronic, a...",autolog serum skin test indic chronic autoimmu...
...,...,...,...,...,...
CAPTUM_Angioedema_Omalizumab_Maoz-Segal 2020.pdf_out_text.txt,Maoz-Segal 2020.pdf,"[Angioedema, Omalizumab]",2020,"[maoz-seg, world, allergi, organ, journal, avv...",maoz-seg world allergi organ journal avv aisce...
CAPTUM_Total IgE_TPO_Sanchez 2019.pdf_out_text.txt,Sanchez 2019.pdf,"[Total IgE, TPO]",2019,"[allergi, asthma, immunol, re, jan, http, piss...",allergi asthma immunol re jan http pissn origi...
CAPTUM_TPO_Omalizumab_Maurer 2011.pdf_out_text.txt,Maurer 2011.pdf,"[TPO, Omalizumab]",2011,"[efficaci, safeti, omalizumab, patient, chroni...",efficaci safeti omalizumab patient chronic urt...
CAPTUM_Antihistamine_Omalizumab_Gimenez Arnau 2016.pdf_out_text.txt,Gimenez Arnau 2016.pdf,"[Antihistamine, Omalizumab]",2016,"[copyright, john, libbey, eurotext, download, ...",copyright john libbey eurotext download nyu la...


In [14]:
print('Tags:')
tags = {element for e in df['Tags'] for element in e}
print(*tags, sep="\n")

Tags:
Angioedema
BAT
ANA
Allergic Diseases
angioedema
Total IgE
anti-FcεRI
Antihistamine
Severity
Thyroglobulin
Anti-IgE
ASST
Omalizumab
IL-24
serverity
Eosinophil
Basophil
CRP
TPO
Basophil count
Cyclosporine
Duration
BHRA
D-Dimer


# Algorithmen anwenden

### TFIDF Vertorizer

In [ ]:
tfidfconvert = TfidfVectorizer(ngram_range=(1,3)).fit(all_words)

In [ ]:
len(tfidfconvert.vocabulary_)

In [ ]:
words_transformed = tfidfconvert.transform(all_words)

### K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

K-Means für mehrere k's berechnen

In [ ]:
Sum_of_squared_distances = []
K = range(1,50)
for k in K: 
    km = KMeans(n_clusters=k)
    km = km.fit(words_transformed)
    Sum_of_squared_distances.append(km.inertia_)
    

In [ ]:
plt.plot(K,Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()